# Demo 1: Simulate the Ricker model and compute EWS

In this demo, we simulate a single stochastic trajectory of the Ricker model to analyse. Alterantively, you could import your own time-series data.

## Simulate the Ricker model

In [1]:
d=2

In [2]:
f=3
d+f

5